In [99]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [106]:
url = "https://business-mail.jp/example"
page = requests.get(url)
soup = BeautifulSoup(page.text, "html")

links = soup.find_all("a")
wants = [
    "https://business-mail.jp/cat_example/outside/",
    "https://business-mail.jp/cat_example/office/",
    "https://business-mail.jp/tag_example/"
]
filtered_links = [a['href'] for a in soup.find_all('a') if any(a['href'].startswith(y) for y in wants)]

data = []

for link in filtered_links:
    # get current topic page
    topic_page = requests.get(link)
    topic_soup = BeautifulSoup(topic_page.text, "html")

    # get all example links from current topic
    topic_example_links = [a["href"] for a in topic_soup.find_all("a") \
        if a["href"].startswith("https://business-mail.jp/example/")]
    topic_example_links = topic_example_links[:-4]
    topic_example_links = [x for x in topic_example_links if x != "https://business-mail.jp/example/"]

    # get title and text from each example
    for example_link in topic_example_links:
        example_page = requests.get(example_link)
        example_soup = BeautifulSoup(example_page.text, "html.parser")
        example_title = example_soup.find_all("h1")[0].get_text()
        example_email = [p.get_text(separator="\n") for p in example_soup.find_all('p')][0]
        data.append([example_link, example_title, example_email])

In [109]:
columns = ["link", "title", "email"]
df = pd.DataFrame(data, columns=columns)

df.to_csv("japanese_business_email_examples.csv")
